In [1]:
!pip install pyspark
!pip install findspark
import findspark
findspark.init() 

from pyspark.sql import SparkSession
# spark.stop()
spark = SparkSession \
.builder \
.config("spark.executor.instances", "1")\
.config('spark.driver.memory','1g')\
.config('spark.executor.memory', '1g') \
.config("spark.memory.offHeap.enabled","true") \
.config("spark.memory.offHeap.size","10g") \
.getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=9b055be141289e03161dc0d16b47fe827d75921ea91f7d6a15cb75ed4901e41e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
user_song = spark.read.csv('/content/gdrive/MyDrive/musicrecom/melodice/user_song_2.csv', inferSchema=True, header=True, nullValue='NA', nanValue='NA',emptyValue='NA').dropna()

In [4]:
user_with_more_listenings = spark.read.csv('/content/gdrive/MyDrive/musicrecom/melodice/user_with_more_listenings.csv', inferSchema=True, header=True, nullValue='NA', nanValue='NA',emptyValue='NA').dropna()

In [6]:
user_song.show()

+-------+---------------+---------------+
|link_id|       cookieid|listening_count|
+-------+---------------+---------------+
|    444|1491698754-3596|              2|
|    444|1494285228-5750|              2|
|    444|1500997338-2939|              1|
|    445|1465152050-1182|              1|
|    445|1465565383-6194|              2|
|    445|1467423780-3721|              1|
|    445|1467592062-7736|              2|
|    445|1469553781-9659|              1|
|    445|1469637113-4090|              1|
|    445|1470757428-2803|              2|
|    445|1470941945-3711|              1|
|    445|1471645750-9603|              1|
|    445|1474206672-9670|              4|
|    445|1475869145-1221|              2|
|    445|1477310230-1177|              2|
|    445|1477343412-6623|              1|
|    445|1477878153-9020|              1|
|    445|1478306200-2182|              1|
|    445|1478967959-8795|              1|
|    445|1479578171-4588|              1|
+-------+---------------+---------

In [10]:
user_with_more_listenings = user_with_more_listenings.withColumnRenamed("cookieid", "cookieid_")

In [12]:
from pyspark.sql.functions import col

user_song_with_more_listenings = user_song.join(user_with_more_listenings, user_song.cookieid == user_with_more_listenings.cookieid_).select("link_id", "cookieid", "listening_count")

In [13]:
user_song_with_more_listenings.show()

+-------+---------------+---------------+
|link_id|       cookieid|listening_count|
+-------+---------------+---------------+
|    444|1491698754-3596|              2|
|    444|1494285228-5750|              2|
|    444|1500997338-2939|              1|
|    445|1465152050-1182|              1|
|    445|1465565383-6194|              2|
|    445|1467423780-3721|              1|
|    445|1467592062-7736|              2|
|    445|1469553781-9659|              1|
|    445|1469637113-4090|              1|
|    445|1470757428-2803|              2|
|    445|1470941945-3711|              1|
|    445|1471645750-9603|              1|
|    445|1474206672-9670|              4|
|    445|1475869145-1221|              2|
|    445|1477310230-1177|              2|
|    445|1477343412-6623|              1|
|    445|1477878153-9020|              1|
|    445|1478306200-2182|              1|
|    445|1478967959-8795|              1|
|    445|1479578171-4588|              1|
+-------+---------------+---------

In [17]:
user_song_with_more_listenings.write.option("header",True).csv("/content/gdrive/MyDrive/musicrecom/melodice/user_song_with_more_listenings.csv")
